### Imports

In [23]:
import fitting_functions as ff
from bokeh.plotting import show
from bokeh.io import output_notebook
from bokeh.models import Range1d
import pymc3 as pm
import matplotlib.pyplot as plt

# !!!
### Write your TIC ID and the Sector

In [24]:
tic_id = 362043085
sector = 5

In [25]:
lightcurve_path = ff.downloading_lightcurve(tic_id, sector)
days, flux = ff.getting_days_and_flux(lightcurve_path)
star_mass, star_radius = ff.getting_star_mass_radius(tic_id)

You're using:  lightcurves/tess2018319095959-s0005-0000000362043085-0125-s_lc.fits
Star mass: 0    1.36
Name: mass, dtype: float64
Star radius: 0    1.71978
Name: rad, dtype: float64


/Users/sishitan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TableReplaceWarning: converted column 'HIP' from integer to float
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/sishitan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TableReplaceWarning: converted column 'KIC' from integer to float
  This is separate from the ipykernel package so we can avoid doing imports until


# !!!
### Select the first and second time you can see the event:

In [26]:
# %matplotlib notebook
# pos = ff.plotting_lightcurve(tic_id, sector, days, flux)


In [27]:
import colorcet as cc

In [28]:
from bokeh.models.callbacks import CustomJS

callback = CustomJS(code="""
// the event that triggered the callback is cb_obj:
// The event type determines the relevant attributes
console.log('Tap event occurred at x-position: ' + cb_obj.x)
""")

tools = "tap,pan,wheel_zoom,box_zoom,reset,previewsave"
plot = figure(title=f"TIC {tic_id} Sector {sector} . Select your t0", tools=tools)

number_for_dots = len(days) // len(cc.bmy) + 1
palette = [cc.bmy[i // number_for_dots] for i in range(len(days))]
plot.xaxis.axis_label = 'Time Since Transit [days]'
plot.yaxis.axis_label = 'Relative Flux [percentage]'
plot.circle(x=days, y=flux, line_color=None,
            fill_color=palette, fill_alpha=0.5, size=10)
# execute a callback whenever the plot canvas is tapped
plot.js_on_event('tap', callback)
show(plot)

In [29]:
# t0_guess = pos[-2][0]
# next_t0_guess = pos[-1][0]
# period_guess = next_t0_guess - t0_guess
# print(f'Your t0 guess is: {t0_guess}')
# print(f'Your next guess is: {next_t0_guess}')
# print(f'Your period guess is: {period_guess}')

t0_guess = 1441.6
period_guess = 6.86
print(f'Your t0 guess is: {t0_guess}')
print(f'Your period guess is: {period_guess}')

Your t0 guess is: 1441.6
Your period guess is: 6.86


### That's your lightcurve folded:

In [30]:
output_notebook()
x_fold = ff.folding(days, t0_guess, period_guess)
p = ff.plotting_folded(days, flux, tic_id, sector, x_fold)
p.x_range = Range1d(-0.4, 0.4)
show(p)

Loading BokehJS ...

### Initial Model

In [ ]:
model, light_curve = ff.manual_fitting(tic_id, sector, days, flux, t0_guess, period_guess, star_radius, star_mass, x_fold)

# Fitting

In [ ]:
trace = ff.best_model(model)

### To see table with fitting parameters

In [ ]:
pm.summary(trace, round_to=5)

### To see graph with fitting parameters, set visual = 'yes'

In [ ]:
median_x_fold, median_t0, median_period, median_radius_ratio, median_impact_parameter = ff.visualizing_fitting_process(days, trace,visual='no')
print(f't0 = {median_t0}')
print(f'period = {median_period}')
print(f'radius_ratio = {median_radius_ratio}')
print(f'impact_parameter = {median_impact_parameter}')

## Final plot


In [ ]:
ff.plotting_final_fit(tic_id, sector, days, flux, trace, model, light_curve, median_x_fold)